In [1]:
import datetime

now = datetime.datetime.now()

In [4]:
now - datetime.timedelta(days=14)

datetime.datetime(2018, 6, 28, 17, 22, 53, 665354)

In [5]:
import pandas as pd
from config.constants import EXCEL_EXPORT_PATH_MERGED, LEAGUES_DATA_DICT, SOLOQ

In [5]:
df = pd.read_excel(LEAGUES_DATA_DICT[SOLOQ][EXCEL_EXPORT_PATH_MERGED])

In [6]:
df['date'] = df.gameCreation.apply(lambda x: x.split(' ')[0])

In [7]:
mad_df = df.loc[df.team_abbv == 'MAD']

## How many games they played

In [8]:
df1 = pd.DataFrame(dict(mad_df.groupby('player_name').count()['currentAccountId']), index=(0,)).T.rename(columns={0: 'games_played'})

## How many times they played a champion per day

In [45]:
champs_practiced = mad_df.groupby(['player_name', 'champ_name']).count()['currentAccountId'].to_frame().reset_index()

In [46]:
champs_practiced.rename(columns={'currentAccountId': 'times_played'}, inplace=True)

In [47]:
df2 = champs_practiced

In [13]:
df2_werlyb = df2.loc[df2.player_name == 'Werlyb']
df2_selfmade = df2.loc[df2.player_name == 'Selfmade']
df2_nemesis = df2.loc[df2.player_name == 'Nemesis']
df2_crownshot = df2.loc[df2.player_name == 'Crownshot']
df2_falco = df2.loc[df2.player_name == 'Falco']

### What champions they did have to practice

In [24]:
dict0 = {'Werlyb': ['Camille', 'Cho\'Gath', 'Fiora', 'Gangplank'], 
         'Selfmade': ['Sejuani', 'Taliyah', 'Nocturne'], 
         'Nemesis': ['Zoe', 'Galio'],
         'Crownshot': ['Swain', 'Mordekaiser', 'Morgana'],
         'Falco': ['Pyke', 'Zyra', 'Zoe', 'Ornn']}

In [79]:
practice_champs = pd.DataFrame([(key, v, 1) for key, values in dict0.items() for v in values]).rename(columns={0: 'player_name', 1: 'champ_name', 2: 'should_train'})

In [82]:
practiced_champs = df2.merge(practice_champs, on=['player_name', 'champ_name'], how='outer').sort_values(['player_name', 'champ_name']).fillna(0)

In [87]:
practiced_champs[['times_played', 'should_train']] = practiced_champs[['times_played', 'should_train']].astype(int)

## How many times they played together

In [96]:
champs_practiced

,player_name,champ_name,times_played
0,Crownshot,Alistar,1
1,Crownshot,Ashe,1
2,Crownshot,Caitlyn,1
3,Crownshot,Ezreal,1
4,Crownshot,Heimerdinger,4
5,Crownshot,Jhin,5
6,Crownshot,Kai'Sa,5
7,Crownshot,Lucian,1
8,Crownshot,Sivir,5
9,Crownshot,Swain,8


In [97]:
dict1 = {}
for name in champs_practiced.player_name.unique():
    df = champs_practiced.loc[(champs_practiced.player_name == name)]
    dict1[name] = {row[1][1]: row[1][2] for row in df.iterrows()}

In [98]:
together = mad_df.groupby(['gameId', 'queueId', 'player_name'], as_index=False).count()

In [99]:
duplicated_ids = together.duplicated('gameId', keep=False)

In [100]:
played_together = together.ix[duplicated_ids]

C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [101]:
from collections import defaultdict

In [102]:
dict2 = defaultdict(int)

In [103]:
for gameid in played_together.gameId.unique():
    df = played_together.loc[played_together.gameId == gameid]
    names = list(df.sort_values('player_name').player_name)
    if len(names) == 2:
        team_name = names[0] + '_' + names[1]
    elif len(names) == 3:
        team_name = names[0] + '_' + names[1] + '_' + names[2]
    elif len(names) == 5:
        team_name = names[0] + '_' + names[1] + '_' + names[2] + '_' + names[3] + '_' + names[4]
    dict2[team_name] += 1

In [104]:
df3 = pd.DataFrame(dict(dict2), index=(0,)).T.rename(columns={0: 'times_played_together'})

In [105]:
df3

,times_played_together
Crownshot_Falco,6
Crownshot_Nemesis,6
Crownshot_Selfmade,1
Falco_Werlyb,1
Nemesis_Selfmade,2
Nemesis_Werlyb,1


## To excel

In [106]:
writer = pd.ExcelWriter('solo_queue_report.xlsx')
# df1.to_excel(writer,'Total games played')
practiced_champs.to_excel(writer,'Champions practiced')
# df2_werlyb.to_excel(writer,'Werlyb champions practiced')
# df2_selfmade.to_excel(writer,'Selfmade champions practiced')
# df2_nemesis.to_excel(writer,'Nemesis champions practiced')
# df2_crownshot.to_excel(writer,'Crownshot champions practiced')
# df2_falco.to_excel(writer,'Falco champions practiced')
df3.to_excel(writer, 'Times played together')
writer.save()